#1. Loading Necessary Libraries

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/Shareddrives/ISEN 613 Final Project/ISEN613-Engineering-Data-Analysis-CourseProject-main"


In [ ]:
# cnn model and accuracy - porosity tensor dataset
# Import packages / libraries

import os
from os.path import dirname, join as pjoin
import scipy.io
import tensorflow as tf
import torch

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random

# baseline model for the classification dataset
import sys
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

!pip install opencv-python
!pip install lime 
!pip install lazypredict

import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from skimage.color import gray2rgb
from skimage.color import rgb2gray
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import inception_v3 as inc_net
from lime import lime_image
import pandas as pd
import glob
from tensorflow.keras.models import load_model
from sklearn.metrics import r2_score
import cv2
from keras.preprocessing.image import ImageDataGenerator
from skimage.segmentation import mark_boundaries

In [ ]:
## LIME ANALYSIS
## Import required libraries / packages

import numpy as np
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import math
import scipy
import scipy.io
from PIL import Image
from scipy import ndimage
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
# import tensorflow_addons as tfa
import pydot
import pydotplus
import graphviz
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
import random
from keras.models import load_model
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import glob
import os
import pandas as pd

import types
from lime.utils.generic_utils import has_arg
from skimage.segmentation import felzenszwalb, slic, quickshift
import copy
from functools import partial

import sklearn
import sklearn.preprocessing
from sklearn.utils import check_random_state
from skimage.color import gray2rgb
from tqdm.auto import tqdm

import scipy.ndimage as ndi
from skimage.segmentation._quickshift_cy import _quickshift_cython

from lime import lime_base
from lime.wrappers.scikit_image import SegmentationAlgorithm

import skimage
from matplotlib import colors
from skimage.segmentation import mark_boundaries, find_boundaries
from skimage.morphology import dilation,square
from collections import Counter
import re
from sklearn.ensemble import RandomForestRegressor



#2. Input Processing
In this section, inputs are loaded into tensor-type variables and converted to numpy-type.

In [ ]:
from numpy.core.fromnumeric import shape
# organize dataset into a numpy structure
# Extract input spectrogram data from .mat files 
# Loading the ground truth xlsx file
gt = pd.read_excel(r"/content/drive/Shareddrives/ISEN 613 Final Project/ISEN613-Engineering-Data-Analysis-CourseProject-main/CNN code and input/Ground Truth - Voxel Tensor Index_ Sample1.xlsx")
perc_area = gt.iloc[0:90, [1,5]]
pore_count = gt.iloc[0:90,[1,2]]

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
# sets print options to 5 decimal places

TENSOR_INPUT_DATA_COMBINED = torch.ones((72,6,129,15))
TENSOR_INPUT_DATA_PRINTING = torch.ones((72,4,129,15))
TENSOR_INPUT_DATA_MILLING = torch.ones((72,2,129,15))
    
    # 72 - Total number of data points for binary classification
    # 6 - Channels (pertaining to each spectrogram)
    # 129  - Freuqency bands for each spectrogram
    # 15 - Time steps

voxel_number = 0

# assign directory
# directory = 'files'
# iterate over files in that directory
# for filename in os.listdir(directory):


directory = (r"/content/drive/Shareddrives/ISEN 613 Final Project/ISEN613-Engineering-Data-Analysis-CourseProject-main/CNN code and input/Tensor_data_72_voxels_sample1")
# Change this directory location to your folder that contains the 72 .mat files from Tensor_data_72_voxels.zip corresponding to Sample 1.
# When working with Sample 4 (Tensor_data_90_voxels_sample4.zip), extract the 90 files in a folder, and update the directory. In addition, 
# update the dimensions in the below for loop (and in other places as required) as each tensor in Sample 4 is of dimension 129x6x4, 
# ie, 129 frequency bands, 6 time steps, 4 spectrograms pertaining to only printing cycle. 

voxel_perc_area = []
voxel_pore_count = []
class_comb_area = np.zeros((72,2))
class_comb_count = np.zeros((72,2))
reg_comb = np.zeros((72,2))

for filename in os.listdir(directory):
    # print(filename)
    mat_fname = pjoin(directory, filename)
    Tensor_data_voxel = scipy.io.loadmat(mat_fname)

    data = list(Tensor_data_voxel. items())
    tensor_array = np.array(data, dtype=object)

    # The spectrogram data is captured in a list from one of the elements of tensor_array
    # Shape of tensor_array is (4,2) and all spectrogram data (129x15x6) is stored in tensor_array[3][1]

    spec_list = list(tensor_array[3][1])
    
    # Extracting the desired output to build the different datasets
    voxel_name = tensor_array[3][0]
    voxel_ind = [int(s) for s in re.findall(r'\d+', voxel_name)][0]
    voxel_perc_area.append(np.array(perc_area.loc[perc_area['Tensor_voxel _index'] == voxel_ind, '%Area Porosity'])[0])
    voxel_pore_count.append(np.array(pore_count.loc[perc_area['Tensor_voxel _index'] == voxel_ind, 'Pore Count'])[0])

    for i in range(len(spec_list)):
        for j in range(15):
            for k in range(6):
                TENSOR_INPUT_DATA_COMBINED[voxel_number,k,i,j] = spec_list[i][j][k]
                    
    TENSOR_INPUT_DATA_PRINTING = TENSOR_INPUT_DATA_COMBINED[:,(1,2,4,5),:,:]
    TENSOR_INPUT_DATA_MILLING = TENSOR_INPUT_DATA_COMBINED[:,(0,3),:,:]                

    voxel_number = voxel_number + 1

# for i in range(72):
#   class_comb_area[i][0] = tensor_np[i]
#   class_comb_count[i][0] = tensor_np[i]
#   reg_comb[i][0] = tensor_np[i]

#   class_comb_area[i][1] = voxel_perc_area[i]
#   class_comb_count[i][1] = voxel_pore_count[i]
#   reg_comb[i][1] = voxel_perc_area[i]


y_target_area = np.array([voxel_perc_area[i] for i in range(72)])
y_target_count = np.array([voxel_pore_count[i] for i in range(72)])

# y_target[4:10] = 1
# y_target[13:18] = 1
# y_target[22:34] = 1
# y_target[37:41] = 1
# y_target[43:46] = 1
# y_target[47] = 1
# y_target[57] = 1
# y_target[59] = 1
# y_target[70] = 1

data_order = np.arange(72)
np.random.shuffle(data_order)

'''
print("---------------------")
print(TENSOR_INPUT_DATA_COMBINED.shape)
print(TENSOR_INPUT_DATA_COMBINED)
print(TENSOR_INPUT_DATA_PRINTING.shape)
print(TENSOR_INPUT_DATA_PRINTING)
print(TENSOR_INPUT_DATA_MILLING.shape)
print(TENSOR_INPUT_DATA_MILLING)
print("---------------------")
print(y_target)
print("---------------------")
data_order = np.arange(72)
np.random.shuffle(data_order)
print(data_order)
'''


'\nprint("---------------------")\nprint(TENSOR_INPUT_DATA_COMBINED.shape)\nprint(TENSOR_INPUT_DATA_COMBINED)\nprint(TENSOR_INPUT_DATA_PRINTING.shape)\nprint(TENSOR_INPUT_DATA_PRINTING)\nprint(TENSOR_INPUT_DATA_MILLING.shape)\nprint(TENSOR_INPUT_DATA_MILLING)\nprint("---------------------")\nprint(y_target)\nprint("---------------------")\ndata_order = np.arange(72)\nnp.random.shuffle(data_order)\nprint(data_order)\n'

In [ ]:
# Saving data as .npy files
NUMPY_INPUT_DATA = TENSOR_INPUT_DATA_COMBINED.numpy()
NUMPY_INPUT_DATA_printing = TENSOR_INPUT_DATA_PRINTING.numpy()
NUMPY_INPUT_DATA_milling = TENSOR_INPUT_DATA_MILLING.numpy()

np.save(r"/content/drive/Shareddrives/ISEN 613 Final Project/ISEN613-Engineering-Data-Analysis-CourseProject-main/CNN code and input/Saved Inputs_Outputs/Comb_Inputs", NUMPY_INPUT_DATA)
np.save(r"/content/drive/Shareddrives/ISEN 613 Final Project/ISEN613-Engineering-Data-Analysis-CourseProject-main/CNN code and input/Saved Inputs_Outputs/Print_Inputs", NUMPY_INPUT_DATA_printing)
np.save(r"/content/drive/Shareddrives/ISEN 613 Final Project/ISEN613-Engineering-Data-Analysis-CourseProject-main/CNN code and input/Saved Inputs_Outputs/Mill_Inputs", NUMPY_INPUT_DATA_milling)
np.save(r"/content/drive/Shareddrives/ISEN 613 Final Project/ISEN613-Engineering-Data-Analysis-CourseProject-main/CNN code and input/Saved Inputs_Outputs/Target_Area", y_target_area)
np.save(r"/content/drive/Shareddrives/ISEN 613 Final Project/ISEN613-Engineering-Data-Analysis-CourseProject-main/CNN code and input/Saved Inputs_Outputs/Target_Count", y_target_count)

# 3. Regression


###3.1. Combined Data

In [ ]:
NUMPY_INPUT_DATA = TENSOR_INPUT_DATA_COMBINED.numpy()

X_train_reg = NUMPY_INPUT_DATA[data_order[0:64],:,:,:]
y_train_reg = y_target_area[data_order[0:64]]
X_test_reg = NUMPY_INPUT_DATA[data_order[64:72],:,:,:]
y_test_reg = y_target_area[data_order[64:72]]
    
# Define the model architecture
model = Sequential()
model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(6, 129, 15)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
#opt = SGD(lr=0.001, momentum=0.9)

model.compile(loss='mean_squared_error', optimizer='adam', metrics='mse')

model.fit(X_train_reg, y_train_reg,
          epochs=100,
          verbose=1,
          validation_data=(X_test_reg, y_test_reg))
score = model.evaluate(X_test_reg, y_test_reg, verbose=0)
print('Model MSE:', score[0])

Epoch 1/100
2/2 [==============================] - 1s 251ms/step - loss: 3.7007 - mse: 3.7007 - val_loss: 0.0819 - val_mse: 0.0819
Epoch 2/100
2/2 [==============================] - 0s 143ms/step - loss: 3.2831 - mse: 3.2831 - val_loss: 0.1453 - val_mse: 0.1453
Epoch 3/100
2/2 [==============================] - 0s 129ms/step - loss: 3.1672 - mse: 3.1672 - val_loss: 0.1701 - val_mse: 0.1701
Epoch 4/100
2/2 [==============================] - 0s 144ms/step - loss: 3.1359 - mse: 3.1359 - val_loss: 0.1778 - val_mse: 0.1778
Epoch 5/100
2/2 [==============================] - 0s 134ms/step - loss: 3.1267 - mse: 3.1267 - val_loss: 0.1803 - val_mse: 0.1803
Epoch 6/100
2/2 [==============================] - 0s 135ms/step - loss: 3.1238 - mse: 3.1238 - val_loss: 0.1813 - val_mse: 0.1813
Epoch 7/100
2/2 [==============================] - 0s 162ms/step - loss: 3.1228 - mse: 3.1228 - val_loss: 0.1817 - val_mse: 0.1817
Epoch 8/100
2/2 [==============================] - 0s 141ms/step - loss: 3.1221 - m

## 3.2. Printing

In [ ]:
NUMPY_INPUT_DATA_printing = TENSOR_INPUT_DATA_PRINTING.numpy()

X_train_reg_printing = NUMPY_INPUT_DATA_printing[data_order[0:64],:,:,:]
y_train_reg = y_target_area[data_order[0:64]]
X_test_reg_printing = NUMPY_INPUT_DATA_printing[data_order[64:72],:,:,:]
y_test_reg = y_target_area[data_order[64:72]]
    
# Define the model architecture
model = Sequential()
model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(4, 129, 15)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
#opt = SGD(lr=0.001, momentum=0.9)

model.compile(loss='mean_squared_error', optimizer='adam', metrics='mse')

model.fit(X_train_reg_printing, y_train_reg,
          epochs=100,
          verbose=1,
          validation_data=(X_test_reg_printing, y_test_reg))
score = model.evaluate(X_test_reg_printing, y_test_reg, verbose=0)
print('Model MSE:', score[0])


Epoch 1/500
2/2 [==============================] - 3s 1s/step - loss: 4.1393 - mse: 4.1393 - val_loss: 2.3610 - val_mse: 2.3610
Epoch 2/500
2/2 [==============================] - 0s 192ms/step - loss: 3.7438 - mse: 3.7438 - val_loss: 2.1643 - val_mse: 2.1643
Epoch 3/500
2/2 [==============================] - 1s 1s/step - loss: 3.4323 - mse: 3.4323 - val_loss: 2.0515 - val_mse: 2.0515
Epoch 4/500
2/2 [==============================] - 2s 2s/step - loss: 3.2496 - mse: 3.2496 - val_loss: 1.9399 - val_mse: 1.9399
Epoch 5/500
2/2 [==============================] - 1s 910ms/step - loss: 3.1326 - mse: 3.1326 - val_loss: 1.8664 - val_mse: 1.8664
Epoch 6/500
2/2 [==============================] - 0s 110ms/step - loss: 3.0513 - mse: 3.0513 - val_loss: 1.8215 - val_mse: 1.8215
Epoch 7/500
2/2 [==============================] - 0s 122ms/step - loss: 2.9969 - mse: 2.9969 - val_loss: 1.7973 - val_mse: 1.7973
Epoch 8/500
2/2 [==============================] - 0s 116ms/step - loss: 2.9657 - mse: 2.965

## 3.3. Milling

In [ ]:
NUMPY_INPUT_DATA_milling = TENSOR_INPUT_DATA_MILLING.numpy()

X_train_reg_milling = NUMPY_INPUT_DATA_milling[data_order[0:64],:,:,:]
y_train_reg = y_target_area[data_order[0:64]]
X_test_reg_milling = NUMPY_INPUT_DATA_milling[data_order[64:72],:,:,:]
y_test_reg = y_target_area[data_order[64:72]]
    
# Define the model architecture
model = Sequential()
model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(2, 129, 15)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
#model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
#opt = SGD(lr=0.001, momentum=0.9)

model.compile(loss='mean_squared_error', optimizer='adam', metrics='mse')

model.fit(X_train_reg_milling, y_train_reg,
          epochs=100,
          verbose=1,
          validation_data=(X_test_reg_milling, y_test_reg))
score = model.evaluate(X_test_reg_milling, y_test_reg, verbose=0)
print('Model MSE:', score[0])


Epoch 1/100
2/2 [==============================] - 1s 274ms/step - loss: 4.2427 - mse: 4.2427 - val_loss: 0.0835 - val_mse: 0.0835
Epoch 2/100
2/2 [==============================] - 0s 86ms/step - loss: 4.1997 - mse: 4.1997 - val_loss: 0.0800 - val_mse: 0.0800
Epoch 3/100
2/2 [==============================] - 0s 61ms/step - loss: 4.1871 - mse: 4.1871 - val_loss: 0.0798 - val_mse: 0.0798
Epoch 4/100
2/2 [==============================] - 0s 61ms/step - loss: 4.1843 - mse: 4.1843 - val_loss: 0.0796 - val_mse: 0.0796
Epoch 5/100
2/2 [==============================] - 0s 70ms/step - loss: 4.1824 - mse: 4.1824 - val_loss: 0.0794 - val_mse: 0.0794
Epoch 6/100
2/2 [==============================] - 0s 78ms/step - loss: 4.1804 - mse: 4.1804 - val_loss: 0.0792 - val_mse: 0.0792
Epoch 7/100
2/2 [==============================] - 0s 76ms/step - loss: 4.1783 - mse: 4.1783 - val_loss: 0.0789 - val_mse: 0.0789
Epoch 8/100
2/2 [==============================] - 0s 69ms/step - loss: 4.1763 - mse: 4.1

# 4. Classification

## 4.1. %Pore Count

In [ ]:
# Specifying the response and converting it to 0,1 type
y_class_count = np.zeros(72)
for i in range(72):
  if y_target_count[i]<=np.median(y_target_count):
    y_class_count[i] = 0
  else: 
    y_class_count[i] = 1

### 4.1.1. Combined

In [ ]:
# CNN Model Architecture - Printing and Milling combined cycles

NUMPY_INPUT_DATA = TENSOR_INPUT_DATA_COMBINED.numpy()


X_train = NUMPY_INPUT_DATA[data_order[0:64],:,:,:]
y_train = y_class_count[data_order[0:64]]
X_test = NUMPY_INPUT_DATA[data_order[64:72],:,:,:]
y_test = y_class_count[data_order[64:72]]


# Define the K-fold Cross Validator
kfold = KFold(n_splits=8, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
train_acc_per_fold = []
train_loss_per_fold = []
val_acc_per_fold = []
val_loss_per_fold = []
test_acc_per_fold = []
test_loss_per_fold = []

for train, val in kfold.split(X_train, y_train):
    
    train_acc = 0
    val_acc = 0
    test_acc = 0
    # Define the model architecture
    while(train_acc < 0.5 or val_acc < 0.5 or test_acc < 0.5):
        model = Sequential()
        model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(6, 129, 15)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        # model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(1, activation='sigmoid'))
    
        # Compile the model
        opt = SGD(lr=0.001, momentum=0.9)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fit data to model
        history = model.fit(X_train[train], y_train[train], epochs=200, verbose=0)
        #train_acc_per_fold.append(history.history['accuracy'])
        #train_loss_per_fold.append(history.history['loss'])
        #summarize_diagnostics(fold_no, history)
        model.save('porosity_bin_model_fold_no_'+ str(fold_no) +'.h5')
    
        # Generate generalization metrics
        train_scores = model.evaluate(X_train[train], y_train[train], verbose=0)
        print(f'Train Score for fold {fold_no}: {model.metrics_names[0]} of {train_scores[0]}; {model.metrics_names[1]} of {train_scores[1]*100}%')
        train_acc = train_scores[1]
        
        val_scores = model.evaluate(X_train[val], y_train[val], verbose=0)
        print(f'Val Score for fold {fold_no}: {model.metrics_names[0]} of {val_scores[0]}; {model.metrics_names[1]} of {val_scores[1]*100}%')
        val_acc = val_scores[1]
        
        test_scores = model.evaluate(X_test, y_test, steps=len(X_test), verbose = 0)
        print(f'Test data Score for fold {fold_no}: {model.metrics_names[0]} of {test_scores[0]}; {model.metrics_names[1]} of {test_scores[1]*100}%')
        test_acc = test_scores[1]
    
    # Increase fold number
    fold_no = fold_no + 1
    train_acc_per_fold.append(train_scores[1] * 100)
    train_loss_per_fold.append(train_scores[0])
    val_acc_per_fold.append(val_scores[1] * 100)
    val_loss_per_fold.append(val_scores[0])
    test_acc_per_fold.append(test_scores[1] * 100)
    test_loss_per_fold.append(test_scores[0])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.6931479573249817; accuracy of 50.0%
Val Score for fold 1: loss of 0.6928583383560181; accuracy of 62.5%
Test data Score for fold 1: loss of 0.693437397480011; accuracy of 37.5%
------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.6931479573249817; accuracy of 50.0%
Val Score for fold 1: loss of 0.6928543448448181; accuracy of 62.5%
Test data Score for fold 1: loss of 0.6934414505958557; accuracy of 37.5%
------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.00482650613412261; accuracy of 100.0%
Val Score for fold 1: loss of 2.584857702255249; accuracy of 37.5%
Test data Score for fold 1: loss of 1.4824942350387573; accuracy of 50.0%
------------------------------------------------------------------------


In [ ]:
LX = NUMPY_INPUT_DATA[data_order,:,:,:]
LY = y_class_count[data_order]

for i in range(1,9):
    model_name = 'porosity_bin_model_fold_no_' + str(i) + '.h5'
    model = load_model(model_name)
    pred = model.predict(LX)
    # LY = np.array([1 if a == 0 else 0 for a in Y])
    pred_class = [0 if a < 0.5 else 1 for a in pred]
    correct_X_ind = list(np.where(pred_class==LY)[0])
    print('------------------------------------------------------------------------')
    print(f'Correct predictions in fold {i} ...')
    print(len(correct_X_ind))  
    pred_test = model.predict(X_test)
    pred_class_test = [0 if a < 0.5 else 1 for a in pred_test]
    correct_X_ind = list(np.where(pred_class_test==y_test)[0])
    print(f'Correct Test data predictions for fold {i} ...')
    print(len(correct_X_ind))

3/3 [==============================] - 0s 21ms/step
------------------------------------------------------------------------
Correct predictions in fold 1 ...
66
1/1 [==============================] - 0s 23ms/step
Correct Test data predictions for fold 1 ...
6
3/3 [==============================] - 0s 21ms/step
------------------------------------------------------------------------
Correct predictions in fold 2 ...
66
1/1 [==============================] - 0s 28ms/step
Correct Test data predictions for fold 2 ...
6
3/3 [==============================] - 0s 21ms/step
------------------------------------------------------------------------
Correct predictions in fold 3 ...
67
1/1 [==============================] - 0s 23ms/step
Correct Test data predictions for fold 3 ...
7
3/3 [==============================] - 0s 21ms/step
------------------------------------------------------------------------
Correct predictions in fold 4 ...
63
1/1 [==============================] - 0s 21ms/step
Cor

### 4.1.2. Printing

In [ ]:
NUMPY_INPUT_DATA_printing = TENSOR_INPUT_DATA_PRINTING.numpy()

X_train_printing = NUMPY_INPUT_DATA_printing[data_order[0:64],:,:,:]
y_train_printing = y_class_count[data_order[0:64]]
X_test_printing = NUMPY_INPUT_DATA_printing[data_order[64:72],:,:,:]
y_test_printing = y_class_count[data_order[64:72]]

# Define the K-fold Cross Validator
kfold = KFold(n_splits=8, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
train_acc_per_fold = []
train_loss_per_fold = []
val_acc_per_fold = []
val_loss_per_fold = []
test_acc_per_fold = []
test_loss_per_fold = []

for train, val in kfold.split(X_train_printing, y_train_printing):
    
    train_acc = 0
    val_acc = 0
    test_acc = 0
    # Define the model architecture
    while(train_acc < 0.5 or val_acc < 0.5 or test_acc < 0.5):
        model = Sequential()
        model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(4, 129, 15)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        # model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(1, activation='sigmoid'))
    
        # Compile the model
        opt = SGD(lr=0.001, momentum=0.9)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fit data to model
        history = model.fit(X_train_printing[train], y_train_printing[train], epochs=200, verbose=0)
        #train_acc_per_fold.append(history.history['accuracy'])
        #train_loss_per_fold.append(history.history['loss'])
        #summarize_diagnostics(fold_no, history)
        model.save('porosity_bin_model_printing_fold_no_'+ str(fold_no) +'.h5')
    
        # Generate generalization metrics
        train_scores = model.evaluate(X_train_printing[train], y_train_printing[train], verbose=0)
        print(f'Train Score for fold {fold_no}: {model.metrics_names[0]} of {train_scores[0]}; {model.metrics_names[1]} of {train_scores[1]*100}%')
        train_acc = train_scores[1]
        
        val_scores = model.evaluate(X_train_printing[val], y_train_printing[val], verbose=0)
        print(f'Val Score for fold {fold_no}: {model.metrics_names[0]} of {val_scores[0]}; {model.metrics_names[1]} of {val_scores[1]*100}%')
        val_acc = val_scores[1]
        
        test_scores = model.evaluate(X_test_printing, y_test_printing, steps=len(X_test), verbose = 0)
        print(f'Test data Score for fold {fold_no}: {model.metrics_names[0]} of {test_scores[0]}; {model.metrics_names[1]} of {test_scores[1]*100}%')
        test_acc = test_scores[1]
    
    # Increase fold number
    fold_no = fold_no + 1
    train_acc_per_fold.append(train_scores[1] * 100)
    train_loss_per_fold.append(train_scores[0])
    val_acc_per_fold.append(val_scores[1] * 100)
    val_loss_per_fold.append(val_scores[0])
    test_acc_per_fold.append(test_scores[1] * 100)
    test_loss_per_fold.append(test_scores[0])


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.014388510026037693; accuracy of 100.0%
Val Score for fold 1: loss of 0.9228906631469727; accuracy of 87.5%
Test data Score for fold 1: loss of 0.5726017951965332; accuracy of 75.0%
------------------------------------------------------------------------
Training for fold 2 ...
Train Score for fold 2: loss of 0.28162404894828796; accuracy of 96.42857313156128%
Val Score for fold 2: loss of 0.8957990407943726; accuracy of 12.5%
Test data Score for fold 2: loss of 0.7168939113616943; accuracy of 37.5%
------------------------------------------------------------------------
Training for fold 2 ...
Train Score for fold 2: loss of 0.018314501270651817; accuracy of 100.0%
Val Score for fold 2: loss of 2.4024858474731445; accuracy of 37.5%
Test data Score for fold 2: loss of 0.9532872438430786; accuracy of 87.5%
-----------------------------------------------------

In [ ]:
LX = NUMPY_INPUT_DATA_printing[data_order,:,:,:]
LY = y_class_count[data_order] 

for i in range(1,9):
    model_name = 'porosity_bin_model_printing_fold_no_' + str(i) + '.h5'
    model = load_model(model_name)
    pred = model.predict(LX)
    # LY = np.array([1 if a == 0 else 0 for a in Y])
    pred_class = [0 if a < 0.5 else 1 for a in pred]
    correct_X_ind = list(np.where(pred_class==LY)[0])
    print('------------------------------------------------------------------------')
    print(f'Correct predictions in fold {i} ...')
    print(len(correct_X_ind))  
    pred_test = model.predict(X_test_printing)
    pred_class_test = [0 if a < 0.5 else 1 for a in pred_test]
    correct_X_ind = list(np.where(pred_class_test==y_test_printing)[0])
    print(f'Correct Test data predictions for fold {i} ...')
    print(len(correct_X_ind))

3/3 [==============================] - 0s 18ms/step
------------------------------------------------------------------------
Correct predictions in fold 1 ...
69
1/1 [==============================] - 0s 24ms/step
Correct Test data predictions for fold 1 ...
6
3/3 [==============================] - 0s 18ms/step
------------------------------------------------------------------------
Correct predictions in fold 2 ...
63
1/1 [==============================] - 0s 24ms/step
Correct Test data predictions for fold 2 ...
6
3/3 [==============================] - 0s 15ms/step
------------------------------------------------------------------------
Correct predictions in fold 3 ...
65
1/1 [==============================] - 0s 27ms/step
Correct Test data predictions for fold 3 ...
6
3/3 [==============================] - 0s 18ms/step
------------------------------------------------------------------------
Correct predictions in fold 4 ...
64
1/1 [==============================] - 0s 19ms/step
Cor

### 4.1.3. Milling

In [ ]:
NUMPY_INPUT_DATA_milling = TENSOR_INPUT_DATA_MILLING.numpy()

X_train_milling = NUMPY_INPUT_DATA_milling[data_order[0:64],:,:,:]
y_train_milling = y_class_count[data_order[0:64]]
X_test_milling = NUMPY_INPUT_DATA_milling[data_order[64:72],:,:,:]
y_test_milling = y_class_count[data_order[64:72]]

# Define the K-fold Cross Validator
kfold = KFold(n_splits=8, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
train_acc_per_fold = []
train_loss_per_fold = []
val_acc_per_fold = []
val_loss_per_fold = []
test_acc_per_fold = []
test_loss_per_fold = []

for train, val in kfold.split(X_train_milling, y_train_milling):
    
    train_acc = 0
    val_acc = 0
    test_acc = 0
    # Define the model architecture
    while(train_acc < 0.5 or val_acc < 0.5 or test_acc < 0.5):
        model = Sequential()
        model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(2, 129, 15)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(MaxPooling2D((2, 2)))
        # model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(1, activation='sigmoid'))
    
        # Compile the model
        opt = SGD(lr=0.001, momentum=0.9)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fit data to model
        history = model.fit(X_train_milling[train], y_train_milling[train], epochs=200, verbose=0)
        #train_acc_per_fold.append(history.history['accuracy'])
        #train_loss_per_fold.append(history.history['loss'])
        #summarize_diagnostics(fold_no, history)
        model.save('porosity_bin_model_milling_fold_no_'+ str(fold_no) +'.h5')
    
        # Generate generalization metrics
        train_scores = model.evaluate(X_train_milling[train], y_train_milling[train], verbose=0)
        print(f'Train Score for fold {fold_no}: {model.metrics_names[0]} of {train_scores[0]}; {model.metrics_names[1]} of {train_scores[1]*100}%')
        train_acc = train_scores[1]
        
        val_scores = model.evaluate(X_train_milling[val], y_train_milling[val], verbose=0)
        print(f'Val Score for fold {fold_no}: {model.metrics_names[0]} of {val_scores[0]}; {model.metrics_names[1]} of {val_scores[1]*100}%')
        val_acc = val_scores[1]
        
        test_scores = model.evaluate(X_test_milling, y_test_milling, steps=len(X_test), verbose = 0)
        print(f'Test data Score for fold {fold_no}: {model.metrics_names[0]} of {test_scores[0]}; {model.metrics_names[1]} of {test_scores[1]*100}%')
        test_acc = test_scores[1]
    
    # Increase fold number
    fold_no = fold_no + 1
    train_acc_per_fold.append(train_scores[1] * 100)
    train_loss_per_fold.append(train_scores[0])
    val_acc_per_fold.append(val_scores[1] * 100)
    val_loss_per_fold.append(val_scores[0])
    test_acc_per_fold.append(test_scores[1] * 100)
    test_loss_per_fold.append(test_scores[0])


------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.6922141313552856; accuracy of 51.78571343421936%
Val Score for fold 1: loss of 0.6956725120544434; accuracy of 37.5%
Test data Score for fold 1: loss of 0.6918460130691528; accuracy of 62.5%
------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.6815966367721558; accuracy of 58.92857313156128%
Val Score for fold 1: loss of 0.695014476776123; accuracy of 50.0%
Test data Score for fold 1: loss of 0.7000432014465332; accuracy of 37.5%
------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.6931476593017578; accuracy of 50.0%
Val Score for fold 1: loss of 0.6929054260253906; accuracy of 62.5%
Test data Score for fold 1: loss of 0.693389892578125; accuracy of 37.5%
-------------------------------------------------

In [ ]:
LX = NUMPY_INPUT_DATA_milling[data_order,:,:,:]
LY = y_class_count[data_order] 

for i in range(1,9):
    model_name = 'porosity_bin_model_milling_fold_no_' + str(i) + '.h5'
    model = load_model(model_name)
    pred = model.predict(LX)
    # LY = np.array([1 if a == 0 else 0 for a in Y])
    pred_class = [0 if a < 0.5 else 1 for a in pred]
    correct_X_ind = list(np.where(pred_class==LY)[0])
    print('------------------------------------------------------------------------')
    print(f'Correct predictions in fold {i} ...')
    print(len(correct_X_ind))  
    pred_test = model.predict(X_test_milling)
    pred_class_test = [0 if a < 0.5 else 1 for a in pred_test]
    correct_X_ind = list(np.where(pred_class_test==y_test_milling)[0])
    print(f'Correct Test data predictions for fold {i} ...')
    print(len(correct_X_ind))

3/3 [==============================] - 0s 9ms/step
------------------------------------------------------------------------
Correct predictions in fold 1 ...
41
1/1 [==============================] - 0s 29ms/step
Correct Test data predictions for fold 1 ...
4
3/3 [==============================] - 0s 8ms/step
------------------------------------------------------------------------
Correct predictions in fold 2 ...
41
1/1 [==============================] - 0s 17ms/step
Correct Test data predictions for fold 2 ...
5
3/3 [==============================] - 0s 9ms/step
------------------------------------------------------------------------
Correct predictions in fold 3 ...
50
1/1 [==============================] - 0s 33ms/step
Correct Test data predictions for fold 3 ...
4
3/3 [==============================] - 0s 8ms/step
------------------------------------------------------------------------
Correct predictions in fold 4 ...
45
1/1 [==============================] - 0s 17ms/step
Correct

## 4.2. %Area Porosity

In [ ]:
y_class_area = np.zeros(72)
for i in range(72):
  if y_target_area[i]<=np.mean(y_target_area):
    y_class_area[i] = 0
  else: 
    y_class_area[i] = 1

[0.000 0.000 0.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 0.000 0.000
 0.000 0.000 0.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000 1.000
 1.000 1.000 1.000 1.000 1.000 1.000 0.000 0.000 0.000 1.000 1.000 1.000
 1.000 0.000 0.000 0.000 0.000 1.000 1.000 1.000 1.000 0.000 0.000 0.000
 0.000 0.000 0.000 1.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 1.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000]


### 4.2.1. Combined

In [ ]:
# CNN Model Architecture - Printing and Milling combined cycles

NUMPY_INPUT_DATA = TENSOR_INPUT_DATA_COMBINED.numpy()


X_train = NUMPY_INPUT_DATA[data_order[0:64],:,:,:]
y_train = y_class_area[data_order[0:64]]
X_test = NUMPY_INPUT_DATA[data_order[64:72],:,:,:]
y_test = y_class_area[data_order[64:72]]


# Define the K-fold Cross Validator
kfold = KFold(n_splits=8, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
train_acc_per_fold = []
train_loss_per_fold = []
val_acc_per_fold = []
val_loss_per_fold = []
test_acc_per_fold = []
test_loss_per_fold = []

for train, val in kfold.split(X_train, y_train):
    
    train_acc = 0
    val_acc = 0
    test_acc = 0
    # Define the model architecture
    while(train_acc < 0.5 or val_acc < 0.5 or test_acc < 0.5):
        model = Sequential()
        model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(6, 129, 15)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        # model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(1, activation='sigmoid'))
    
        # Compile the model
        opt = SGD(lr=0.001, momentum=0.9)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fit data to model
        history = model.fit(X_train[train], y_train[train], epochs=200, verbose=0)
        #train_acc_per_fold.append(history.history['accuracy'])
        #train_loss_per_fold.append(history.history['loss'])
        #summarize_diagnostics(fold_no, history)
        model.save('porosity_bin_model_fold_no_'+ str(fold_no) +'.h5')
    
        # Generate generalization metrics
        train_scores = model.evaluate(X_train[train], y_train[train], verbose=0)
        print(f'Train Score for fold {fold_no}: {model.metrics_names[0]} of {train_scores[0]}; {model.metrics_names[1]} of {train_scores[1]*100}%')
        train_acc = train_scores[1]
        
        val_scores = model.evaluate(X_train[val], y_train[val], verbose=0)
        print(f'Val Score for fold {fold_no}: {model.metrics_names[0]} of {val_scores[0]}; {model.metrics_names[1]} of {val_scores[1]*100}%')
        val_acc = val_scores[1]
        
        test_scores = model.evaluate(X_test, y_test, steps=len(X_test), verbose = 0)
        print(f'Test data Score for fold {fold_no}: {model.metrics_names[0]} of {test_scores[0]}; {model.metrics_names[1]} of {test_scores[1]*100}%')
        test_acc = test_scores[1]
    
    # Increase fold number
    fold_no = fold_no + 1
    train_acc_per_fold.append(train_scores[1] * 100)
    train_loss_per_fold.append(train_scores[0])
    val_acc_per_fold.append(val_scores[1] * 100)
    val_loss_per_fold.append(val_scores[0])
    test_acc_per_fold.append(test_scores[1] * 100)
    test_loss_per_fold.append(test_scores[0])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.6931479573249817; accuracy of 50.0%
Val Score for fold 1: loss of 0.6928583383560181; accuracy of 62.5%
Test data Score for fold 1: loss of 0.693437397480011; accuracy of 37.5%
------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.6931479573249817; accuracy of 50.0%
Val Score for fold 1: loss of 0.6928543448448181; accuracy of 62.5%
Test data Score for fold 1: loss of 0.6934414505958557; accuracy of 37.5%
------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.00482650613412261; accuracy of 100.0%
Val Score for fold 1: loss of 2.584857702255249; accuracy of 37.5%
Test data Score for fold 1: loss of 1.4824942350387573; accuracy of 50.0%
------------------------------------------------------------------------


In [ ]:
LX = NUMPY_INPUT_DATA[data_order,:,:,:]
LY = y_class_area[data_order]

for i in range(1,9):
    model_name = 'porosity_bin_model_fold_no_' + str(i) + '.h5'
    model = load_model(model_name)
    pred = model.predict(LX)
    # LY = np.array([1 if a == 0 else 0 for a in Y])
    pred_class = [0 if a < 0.5 else 1 for a in pred]
    correct_X_ind = list(np.where(pred_class==LY)[0])
    print('------------------------------------------------------------------------')
    print(f'Correct predictions in fold {i} ...')
    print(len(correct_X_ind))  
    pred_test = model.predict(X_test)
    pred_class_test = [0 if a < 0.5 else 1 for a in pred_test]
    correct_X_ind = list(np.where(pred_class_test==y_test)[0])
    print(f'Correct Test data predictions for fold {i} ...')
    print(len(correct_X_ind))

### 4.2.2. Printing

In [ ]:
NUMPY_INPUT_DATA_printing = TENSOR_INPUT_DATA_PRINTING.numpy()

X_train_printing = NUMPY_INPUT_DATA_printing[data_order[0:64],:,:,:]
y_train_printing = y_class_area[data_order[0:64]]
X_test_printing = NUMPY_INPUT_DATA_printing[data_order[64:72],:,:,:]
y_test_printing = y_class_area[data_order[64:72]]

# Define the K-fold Cross Validator
kfold = KFold(n_splits=8, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
train_acc_per_fold = []
train_loss_per_fold = []
val_acc_per_fold = []
val_loss_per_fold = []
test_acc_per_fold = []
test_loss_per_fold = []

for train, val in kfold.split(X_train_printing, y_train_printing):
    
    train_acc = 0
    val_acc = 0
    test_acc = 0
    # Define the model architecture
    while(train_acc < 0.5 or val_acc < 0.5 or test_acc < 0.5):
        model = Sequential()
        model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(4, 129, 15)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        # model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(1, activation='sigmoid'))
    
        # Compile the model
        opt = SGD(lr=0.001, momentum=0.9)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fit data to model
        history = model.fit(X_train_printing[train], y_train_printing[train], epochs=200, verbose=0)
        #train_acc_per_fold.append(history.history['accuracy'])
        #train_loss_per_fold.append(history.history['loss'])
        #summarize_diagnostics(fold_no, history)
        model.save('porosity_bin_model_printing_fold_no_'+ str(fold_no) +'.h5')
    
        # Generate generalization metrics
        train_scores = model.evaluate(X_train_printing[train], y_train_printing[train], verbose=0)
        print(f'Train Score for fold {fold_no}: {model.metrics_names[0]} of {train_scores[0]}; {model.metrics_names[1]} of {train_scores[1]*100}%')
        train_acc = train_scores[1]
        
        val_scores = model.evaluate(X_train_printing[val], y_train_printing[val], verbose=0)
        print(f'Val Score for fold {fold_no}: {model.metrics_names[0]} of {val_scores[0]}; {model.metrics_names[1]} of {val_scores[1]*100}%')
        val_acc = val_scores[1]
        
        test_scores = model.evaluate(X_test_printing, y_test_printing, steps=len(X_test), verbose = 0)
        print(f'Test data Score for fold {fold_no}: {model.metrics_names[0]} of {test_scores[0]}; {model.metrics_names[1]} of {test_scores[1]*100}%')
        test_acc = test_scores[1]
    
    # Increase fold number
    fold_no = fold_no + 1
    train_acc_per_fold.append(train_scores[1] * 100)
    train_loss_per_fold.append(train_scores[0])
    val_acc_per_fold.append(val_scores[1] * 100)
    val_loss_per_fold.append(val_scores[0])
    test_acc_per_fold.append(test_scores[1] * 100)
    test_loss_per_fold.append(test_scores[0])


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.014388510026037693; accuracy of 100.0%
Val Score for fold 1: loss of 0.9228906631469727; accuracy of 87.5%
Test data Score for fold 1: loss of 0.5726017951965332; accuracy of 75.0%
------------------------------------------------------------------------
Training for fold 2 ...
Train Score for fold 2: loss of 0.28162404894828796; accuracy of 96.42857313156128%
Val Score for fold 2: loss of 0.8957990407943726; accuracy of 12.5%
Test data Score for fold 2: loss of 0.7168939113616943; accuracy of 37.5%
------------------------------------------------------------------------
Training for fold 2 ...
Train Score for fold 2: loss of 0.018314501270651817; accuracy of 100.0%
Val Score for fold 2: loss of 2.4024858474731445; accuracy of 37.5%
Test data Score for fold 2: loss of 0.9532872438430786; accuracy of 87.5%
-----------------------------------------------------

In [ ]:
LX = NUMPY_INPUT_DATA_printing[data_order,:,:,:]
LY = y_class_area[data_order] 

for i in range(1,9):
    model_name = 'porosity_bin_model_printing_fold_no_' + str(i) + '.h5'
    model = load_model(model_name)
    pred = model.predict(LX)
    # LY = np.array([1 if a == 0 else 0 for a in Y])
    pred_class = [0 if a < 0.5 else 1 for a in pred]
    correct_X_ind = list(np.where(pred_class==LY)[0])
    print('------------------------------------------------------------------------')
    print(f'Correct predictions in fold {i} ...')
    print(len(correct_X_ind))  
    pred_test = model.predict(X_test_printing)
    pred_class_test = [0 if a < 0.5 else 1 for a in pred_test]
    correct_X_ind = list(np.where(pred_class_test==y_test_printing)[0])
    print(f'Correct Test data predictions for fold {i} ...')
    print(len(correct_X_ind))

3/3 [==============================] - 0s 18ms/step
------------------------------------------------------------------------
Correct predictions in fold 1 ...
69
1/1 [==============================] - 0s 24ms/step
Correct Test data predictions for fold 1 ...
6
3/3 [==============================] - 0s 18ms/step
------------------------------------------------------------------------
Correct predictions in fold 2 ...
63
1/1 [==============================] - 0s 24ms/step
Correct Test data predictions for fold 2 ...
6
3/3 [==============================] - 0s 15ms/step
------------------------------------------------------------------------
Correct predictions in fold 3 ...
65
1/1 [==============================] - 0s 27ms/step
Correct Test data predictions for fold 3 ...
6
3/3 [==============================] - 0s 18ms/step
------------------------------------------------------------------------
Correct predictions in fold 4 ...
64
1/1 [==============================] - 0s 19ms/step
Cor

### 4.2.3. Milling

In [ ]:
NUMPY_INPUT_DATA_milling = TENSOR_INPUT_DATA_MILLING.numpy()

X_train_milling = NUMPY_INPUT_DATA_milling[data_order[0:64],:,:,:]
y_train_milling = y_class_area[data_order[0:64]]
X_test_milling = NUMPY_INPUT_DATA_milling[data_order[64:72],:,:,:]
y_test_milling = y_class_area[data_order[64:72]]

# Define the K-fold Cross Validator
kfold = KFold(n_splits=8, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
train_acc_per_fold = []
train_loss_per_fold = []
val_acc_per_fold = []
val_loss_per_fold = []
test_acc_per_fold = []
test_loss_per_fold = []

for train, val in kfold.split(X_train_milling, y_train_milling):
    
    train_acc = 0
    val_acc = 0
    test_acc = 0
    # Define the model architecture
    while(train_acc < 0.5 or val_acc < 0.5 or test_acc < 0.5):
        model = Sequential()
        model.add(Conv2D(6, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(2, 129, 15)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(1, (7, 7), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(MaxPooling2D((2, 2)))
        # model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        # model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(1, activation='sigmoid'))
    
        # Compile the model
        opt = SGD(lr=0.001, momentum=0.9)
        model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        # Fit data to model
        history = model.fit(X_train_milling[train], y_train_milling[train], epochs=200, verbose=0)
        #train_acc_per_fold.append(history.history['accuracy'])
        #train_loss_per_fold.append(history.history['loss'])
        #summarize_diagnostics(fold_no, history)
        model.save('porosity_bin_model_milling_fold_no_'+ str(fold_no) +'.h5')
    
        # Generate generalization metrics
        train_scores = model.evaluate(X_train_milling[train], y_train_milling[train], verbose=0)
        print(f'Train Score for fold {fold_no}: {model.metrics_names[0]} of {train_scores[0]}; {model.metrics_names[1]} of {train_scores[1]*100}%')
        train_acc = train_scores[1]
        
        val_scores = model.evaluate(X_train_milling[val], y_train_milling[val], verbose=0)
        print(f'Val Score for fold {fold_no}: {model.metrics_names[0]} of {val_scores[0]}; {model.metrics_names[1]} of {val_scores[1]*100}%')
        val_acc = val_scores[1]
        
        test_scores = model.evaluate(X_test_milling, y_test_milling, steps=len(X_test), verbose = 0)
        print(f'Test data Score for fold {fold_no}: {model.metrics_names[0]} of {test_scores[0]}; {model.metrics_names[1]} of {test_scores[1]*100}%')
        test_acc = test_scores[1]
    
    # Increase fold number
    fold_no = fold_no + 1
    train_acc_per_fold.append(train_scores[1] * 100)
    train_loss_per_fold.append(train_scores[0])
    val_acc_per_fold.append(val_scores[1] * 100)
    val_loss_per_fold.append(val_scores[0])
    test_acc_per_fold.append(test_scores[1] * 100)
    test_loss_per_fold.append(test_scores[0])


------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.6922141313552856; accuracy of 51.78571343421936%
Val Score for fold 1: loss of 0.6956725120544434; accuracy of 37.5%
Test data Score for fold 1: loss of 0.6918460130691528; accuracy of 62.5%
------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.6815966367721558; accuracy of 58.92857313156128%
Val Score for fold 1: loss of 0.695014476776123; accuracy of 50.0%
Test data Score for fold 1: loss of 0.7000432014465332; accuracy of 37.5%
------------------------------------------------------------------------
Training for fold 1 ...
Train Score for fold 1: loss of 0.6931476593017578; accuracy of 50.0%
Val Score for fold 1: loss of 0.6929054260253906; accuracy of 62.5%
Test data Score for fold 1: loss of 0.693389892578125; accuracy of 37.5%
-------------------------------------------------

In [ ]:
LX = NUMPY_INPUT_DATA_milling[data_order,:,:,:]
LY = y_class_area[data_order] 

for i in range(1,9):
    model_name = 'porosity_bin_model_milling_fold_no_' + str(i) + '.h5'
    model = load_model(model_name)
    pred = model.predict(LX)
    # LY = np.array([1 if a == 0 else 0 for a in Y])
    pred_class = [0 if a < 0.5 else 1 for a in pred]
    correct_X_ind = list(np.where(pred_class==LY)[0])
    print('------------------------------------------------------------------------')
    print(f'Correct predictions in fold {i} ...')
    print(len(correct_X_ind))  
    pred_test = model.predict(X_test_milling)
    pred_class_test = [0 if a < 0.5 else 1 for a in pred_test]
    correct_X_ind = list(np.where(pred_class_test==y_test_milling)[0])
    print(f'Correct Test data predictions for fold {i} ...')
    print(len(correct_X_ind))

3/3 [==============================] - 0s 9ms/step
------------------------------------------------------------------------
Correct predictions in fold 1 ...
41
1/1 [==============================] - 0s 29ms/step
Correct Test data predictions for fold 1 ...
4
3/3 [==============================] - 0s 8ms/step
------------------------------------------------------------------------
Correct predictions in fold 2 ...
41
1/1 [==============================] - 0s 17ms/step
Correct Test data predictions for fold 2 ...
5
3/3 [==============================] - 0s 9ms/step
------------------------------------------------------------------------
Correct predictions in fold 3 ...
50
1/1 [==============================] - 0s 33ms/step
Correct Test data predictions for fold 3 ...
4
3/3 [==============================] - 0s 8ms/step
------------------------------------------------------------------------
Correct predictions in fold 4 ...
45
1/1 [==============================] - 0s 17ms/step
Correct